Blast, I was hoping not to have to generate the whole grid, but I think I'm going to have to. Having said that, I don't think it'll be too big, they haven't asked for a target value of a gazillion or whatever.

So I'm going to represent the grid as a dictionary mapping tuples to values. That's really inefficient, but I don't think that the grid's going to be big enough to cause a problem. So that'll look like:

`{(0,0):1, (1,0):1, (1,1):2, ... }`

Start with a function that will sum the adjacent squares:

In [1]:
def sum_adjacent_squares(coordsIn, gridIn):
    '''Return the sum of all adjacent squares to coordsIn from gridIn'''
    x=coordsIn[0]
    y=coordsIn[1]
    
    return sum([gridIn.get((x-1, y-1), 0),
                gridIn.get((x-1, y), 0),
                gridIn.get((x-1, y+1), 0),
                gridIn.get((x, y-1), 0),
                gridIn.get((x, y+1), 0),
                gridIn.get((x+1, y-1), 0),
                gridIn.get((x+1, y), 0),
                gridIn.get((x+1, y+1), 0)])

# Note that coordinates will usually be sent as a list
sum_adjacent_squares([1, 0], {(0,0):1, (0,1):1, (1,1):2})

4

OK, so now follow the path, using the code from the previous notebook

In [2]:
def turn(dirIn_str):
    '''Returns the direction you're facing after changing direction'''
    assert dirIn_str in ['left', 'right', 'up', 'down'], "Incorrect value: {}".format(dirIn_str)
    if dirIn_str=='left':
        return 'down'
    if dirIn_str=='down':
        return 'right'
    if dirIn_str=='right':
        return 'up'
    if dirIn_str=='up':
        return 'left'
    
turn('left')

'down'

Now let's alter the code so that it returns the grid after n steps:

In [3]:

def take_n_steps(target_i):
    '''Return the coordinates after taking target_i steps, starting at 1'''

    coords=[0,0]
    facing_str='right'
    currentPathLength_i=1
    increasePathLength_bool=False
    square_i=1
    gridOut_dict={(0,0):1}

    # Follow the path until we've done n steps:

    distAlongCurrentPath_i=0
    while target_i > 1:  # This computing; who the hell starts at 1??
        
        # Move 1 step in the direction facing:
        if facing_str=='right':
            coords[0]+=1
        elif facing_str=='up':
            coords[1]+=1
        elif facing_str=='left':
            coords[0]-=1
        elif facing_str=='down':
            coords[1]-=1
        distAlongCurrentPath_i+=1
        
        # If we're at the end of the current path,
        if distAlongCurrentPath_i==currentPathLength_i:
            # Turn
            facing_str=turn(facing_str)
            # Start a new path
            distAlongCurrentPath_i=0
            # And increase the path length
            if increasePathLength_bool:
                currentPathLength_i+=1
            # on alternate paths.
            increasePathLength_bool=not increasePathLength_bool
            
        # Now update the output grid with its current values:
        gridOut_dict[(coords[0], coords[1])]=sum_adjacent_squares(coords,
                                                                  gridOut_dict)
        
        
        target_i-=1
        
    return gridOut_dict
                
take_n_steps(12)
    

{(-1, -1): 11,
 (-1, 0): 10,
 (-1, 1): 5,
 (0, -1): 23,
 (0, 0): 1,
 (0, 1): 4,
 (1, -1): 25,
 (1, 0): 1,
 (1, 1): 2,
 (2, -1): 26,
 (2, 0): 54,
 (2, 1): 57}

Good, that seems to match the test case. Now let's do a quick hack to stop when we hit a bigger value than the target:

In [4]:

def step_until_target(target_i):
    '''Return the coordinates after taking target_i steps, starting at 1'''

    coords=[0,0]
    facing_str='right'
    currentPathLength_i=1
    increasePathLength_bool=False
    square_i=1
    gridOut_dict={(0,0):1}

    # Follow the path until we've done n steps:

    distAlongCurrentPath_i=0
    while True:  # We'll break at the right moment. Not 
                 # the best practice, but never mind.
        
        # Move 1 step in the direction facing:
        if facing_str=='right':
            coords[0]+=1
        elif facing_str=='up':
            coords[1]+=1
        elif facing_str=='left':
            coords[0]-=1
        elif facing_str=='down':
            coords[1]-=1
        distAlongCurrentPath_i+=1
        
        # If we're at the end of the current path,
        if distAlongCurrentPath_i==currentPathLength_i:
            # Turn
            facing_str=turn(facing_str)
            # Start a new path
            distAlongCurrentPath_i=0
            # And increase the path length
            if increasePathLength_bool:
                currentPathLength_i+=1
            # on alternate paths.
            increasePathLength_bool=not increasePathLength_bool
            
        # Now update the output grid with its current values:
        currentValue_i=sum_adjacent_squares(coords, gridOut_dict)
        if currentValue_i > target_i:
            break
        gridOut_dict[(coords[0], coords[1])]=sum_adjacent_squares(coords,
                                                                  gridOut_dict)

    return currentValue_i
                
step_until_target(145)
    

147

Seems OK... Now for the test case:

In [5]:
step_until_target(289326)

295229

Hoorah!